In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

# 1) load data
t_exp,   I_exp   = np.loadtxt('JS_A1_5_1.xy',   unpack=True)
t_ideal, I_ideal, z = np.loadtxt('MnO_rocksalt.xy', unpack=True)
sigma = np.sqrt(I_exp); sigma[sigma==0]=1.0

# 2) define a Gaussian generator from delta‐peaks
def make_profile(t, peaks, ints, width):
    # peaks: 1D array of delta‐peak positions
    # ints:  1D array of their intensities
    # width: Gaussian sigma (in same units as t)
    # returns: continuous profile sampled at t
    d = (t[:,None] - peaks[None,:]) / width
    G = np.exp(-0.5 * d**2)           # unnormalized Gaussians
    # you can normalize by (width*sqrt(2π)) if you care about absolute areas
    return (G * ints[None,:]).sum(axis=1)

# 3) residual function: shift, scale, width
def residuals(p):
    shift, scale, w = p
    profile = make_profile(t_exp - shift, t_ideal, I_ideal, w)
    model = scale * profile
    return (model - I_exp) / sigma

# 4) initial guesses + bounds
p0 = [0.0, 1.0, 0.05]                     # no shift, unit scale, narrow sigma
bds = ([-0.5, 0.0, 0.001],                # shift ±0.5°, scale ≥0, σ ≥1 mdeg
       [ 0.5, np.inf, 0.5])               # width ≤0.5°
res = least_squares(residuals, p0, bounds=bds)

shift_opt, scale_opt, w_opt = res.x
red_chi2 = np.sum(res.fun**2)/(len(res.fun)-len(p0))
print(f"Shift = {shift_opt:.4f}°,  Scale = {scale_opt:.4f},  σ_gauss = {w_opt:.4f}°")
print(f"Reduced χ² = {red_chi2:.2f}")

# 5) build final fitted curve
I_fit = scale_opt * make_profile(t_exp - shift_opt, t_ideal, I_ideal, w_opt)

# 6) plot
plt.figure(figsize=(8,4))
plt.plot(t_exp, I_exp, 'k-',     label='I(obs)')
plt.plot(t_exp, I_fit, 'r--',    label='I(calc)')
plt.plot(t_exp, I_exp-I_fit, 'b-', label='Obs−Calc', alpha=0.6)
plt.xlabel('2θ (°)'); plt.ylabel('Intensity')
plt.legend(); plt.tight_layout()
plt.show()


In [ ]:
# cell 1 – setup + data‐load + quick check
%matplotlib inline

import os
import numpy  as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize   import least_squares

# point this at your real folder
os.chdir(r'C:\Users\dougs\XRD_Plotter')  
print("CWD:", os.getcwd())

# load
t_exp, I_exp   = np.loadtxt('JS_A1_5_1.xy',   unpack=True)
t_ideal, I_ideal = np.loadtxt('MnO_rocksalt.xy', unpack=True)

# quick sanity checks
print("exp_pattern:  n=", len(t_exp), "   2θ range =", t_exp.min(), "→", t_exp.max())
print("ideal_pattern:", len(t_ideal), " points,  2θ span:", t_ideal.min(), "→", t_ideal.max())

# plot raw data just to be sure they’re what you think
plt.figure(figsize=(6,3))
plt.plot(t_exp,   I_exp,   'k-', label='exp')
plt.plot(t_ideal, I_ideal, 'r.', label='ideal (raw)')
plt.legend()
plt.title("Raw XY data")
plt.show()
